#### Package Import

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Nvidia Holder Analysis - Querying Data.ipynb"')))))
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Nvidia Holder Analysis - Querying Data.ipynb"')))))
import apis.pa.mypaengine as pa
import apis.qe.myqengine as qe
import warnings
warnings.filterwarnings("ignore")

Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 5


### Screening for IDs & Characteristics using Quant Engine

#### 1. Get All Mutual Funds that Hold Nvidia

##### a. Query data using FQL

In [31]:
#set universe to NVDA-US
univ = qe.IdUniverse(ids = ['NVDA-US'],
                    universe_type ='Equity')

#set to latest month end
time_series = qe.TimeSeries(start_date='-11M',
                            end_date = '0M')

#Define FQL Formulas
formulas = ['OS_TOP_HLDR_ID(ALL,#DATE,,M,,M)','OS_TOP_HLDR_NAME(ALL,#DATE,,M,,M,,"EN")','OS_TOP_HLDR_MV(ALL,#DATE,,M,,M,SEC,USD)']

#Calculate
q_req = qe.calculate(universe=univ, dates = time_series, formulas=formulas,is_array_return_type=True,source= 'FqlExpression')


##### b. Data prep

In [32]:
df_temp = q_req.data.copy()

#clean up table
df_temp.rename(columns ={'OS_TOP_HLDR_ID(ALL,#DATE,,M,,M)':'hldr_id',
                    'OS_TOP_HLDR_NAME(ALL,#DATE,,M,,M,,"EN")':'hldr_name',
                    'OS_TOP_HLDR_MV(ALL,#DATE,,M,,M,SEC,USD)':'position_mv',
                    },inplace=True)
df_temp = df_temp.set_index(['DATE'])
                    

#Expand arrays and rejoin
df = pd.concat([df_temp['hldr_id'].explode() ,df_temp['hldr_name'].explode(),df_temp['position_mv'].explode()],axis=1)
df=df.reset_index().set_index(['DATE','hldr_id'])
df.head()

hldr_name    position_mv
DATE     hldr_id                                                 
20220930 M4004630  Vanguard Total Stock Market ETF  8608433637.51
         M4004543          Vanguard 500 Index Fund  6887581441.98
         M4017554                Invesco QQQ Trust  3764864600.41
         M4006564           SPDR S&P 500 ETF Trust  3291663073.53
         M4001597          Fidelity 500 Index Fund  3283661773.07

#### 2. Get exchange ticker and other additional data items for holder IDs returned

##### a. Query data using FQL

In [9]:
#Set universe to hldr_ids from previous step
fund_univ = qe.IdUniverse(ids = df.index.get_level_values(1).unique().to_list(),universe_type ='Equity')

#Set Formulas using Holder Formulas
formulas = ['OS_HLDR_MSTYLE','OS_HLDR_MF_FAMILY','STRING(FIRST_ITEM_AV(OS_FUND_TICKER))','FFD_AUM(#DATE,,M,USD)']

#Calculate
q_req = qe.calculate(universe=fund_univ, dates = time_series, formulas=formulas,source= 'FqlExpression')

In [30]:
#Clean up table
dff_arc = q_req.data.copy()

In [39]:
dff = dff_arc.copy()

dff.rename(columns= {"UNIVERSE":"hldr_id",
                    'OS_HLDR_MSTYLE':'style',
                    'OS_HLDR_MF_FAMILY':'fund_family',
                    'STRING(FIRST_ITEM_AV(OS_FUND_TICKER))':'ticker',
                    'FFD_AUM(#DATE,,M,USD)':'aum',
                    },inplace=True)
dff.set_index(['DATE','hldr_id'],inplace=True)
dff

style            fund_family  ticker           aum
DATE     hldr_id                                                           
20220930 M4004630   Generalist         Vanguard Funds   VTSMX  1.058553e+12
         M4004543   Generalist         Vanguard Funds   VFINX  6.873636e+11
         M4017554   Generalist      PowerShares Funds     QQQ  1.482319e+11
         M4006564   Generalist             SPDR Funds     SPY  3.274303e+11
         M4001597        Index                    @NA   FXAIX  3.254562e+11
...                        ...                    ...     ...           ...
20230831 M26129734  Generalist                    @NA    N100  1.329879e+06
         M4257447   Generalist                    @NA    CBLS  5.075026e+06
         M23431828  Generalist                    @NA  440340  1.967357e+07
         M26189410      Growth         Congress Funds    CAML  1.296647e+06
         M21374403  Generalist  Smart Sentiment Funds    OAIE  5.891182e+05

[79632 rows x 4 columns]

##### b. Data prep

In [40]:

#join datasets
dff = dff.join(df,how='left')
#calculate 
dff['weight_in_nvda'] =dff['position_mv'] /dff['aum']

#Replace @NA Groupings with Other
dff['fund_family']=dff['fund_family'].replace('@NA','Other')
dff['style']=dff['style'].replace('@NA','Other')
dff = dff[dff['ticker']!='@NA']
#Clean up table
dff[dff['ticker']!='']
dff = dff.replace('',np.nan)
dff = dff.dropna()


#preview
dff.head()

style        fund_family ticker           aum  \
DATE     hldr_id                                                        
20220930 M4004630  Generalist     Vanguard Funds  VTSMX  1.058553e+12   
         M4004543  Generalist     Vanguard Funds  VFINX  6.873636e+11   
         M4017554  Generalist  PowerShares Funds    QQQ  1.482319e+11   
         M4006564  Generalist         SPDR Funds    SPY  3.274303e+11   
         M4001597       Index              Other  FXAIX  3.254562e+11   

                                         hldr_name   position_mv  \
DATE     hldr_id                                                   
20220930 M4004630  Vanguard Total Stock Market ETF  8.608434e+09   
         M4004543          Vanguard 500 Index Fund  6.887581e+09   
         M4017554                Invesco QQQ Trust  3.764865e+09   
         M4006564           SPDR S&P 500 ETF Trust  3.291663e+09   
         M4001597          Fidelity 500 Index Fund  3.283662e+09   

                   weight_in_nvda  
DATE     hldr_id                   
20220930 M4004630        0.008132  
         M4004543        0.010020  
         M4017554        0.025398  
         M4006564        0.010053  
         M4001597        0.010089

In [41]:
dff.to_pickle('../../data/Nvidia Holder Analysis/screening_full_universe_v2.pkl')